In [ ]:
import pynwb
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

from dandi.download import download as dandi_download
import os

## Download a few files

1. Visit https://dandiarchive.org/dandiset/000009/draft

2. Click on the folder navigation icon

3. Enter subject directory: `sub-anm00229327`

4. Copy the URL and paste it below to download with the dandi client

In [ ]:
url = "https://gui.dandiarchive.org/#/folder/5e72b6ac3da50caa9adb0498"

You can retrieve a file programmatically

In [ ]:
dandi_download([url], os.getcwd(), existing="skip", develop_debug=False)

Or using the shell. DANDI will check if the same file exists.

In [ ]:
%%bash -s "$url"
dandi download $1

#### Set the filename to one of the download files

In [ ]:
file_name = 'sub-anm00229327/sub-anm00229327_ses-20170627T093438_ecephys.nwb'

### Use NWB to read the file

In [ ]:
nwb_io = NWBHDF5IO(file_name, mode='r')
nwb = nwb_io.read()

### Use the NWBWidget to introspect it

In [ ]:
nwb2widget(nwb)